# Homework 5 - Visit the Wikipedia hyperlinks graph!
## Group #18
### Giorgio Zannini Quirini - Roberta Passarelli 

Our goal is to perform an analysis of the **Wikipedia Hyperlink graph**: we will rank the pages knowing that they belong to a certain category.

## Research questions
## [RQ1]
Building the graph $G = (V, E)$, where:
- $V$ is the set of articles
- $E$ the hyperlinks among them.

In [1]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm
import numpy as np
import statistics
from collections import Counter 
from operator import itemgetter

First of all, we read the file `wiki-topcats-reduced.txt`.

In [2]:
reduced = pd.read_csv('wiki-topcats-reduced.txt', sep = '\t', header = None)
reduced.head()

,0,1
0,52,401135
1,52,1069112
2,52,1163551
3,62,12162
4,62,167659


We create two dictionaries: `graph1` with the following structure:
- key: the number of vertice of departure in the column `0`;
- value: the number of vertice of arrival in the column `1`;

and `graph2` with the following structure:
- key: the number of vertice of departure in the column `1`;
- value: the number of vertice of arrival in the column `0`.

In [3]:
graph1 = {}
graph2 = {}
for node_1_2 in reduced.values:
    if node_1_2[0] not in graph1:
        graph1[node_1_2[0]] = [node_1_2[1]]
    else:
        graph1[node_1_2[0]].append(node_1_2[1])
        
    if node_1_2[1] not in graph2:
        graph2[node_1_2[1]] = [node_1_2[0]]
    else:
        graph2[node_1_2[1]].append(node_1_2[0])

- Graph is **directed**.

A graph is undirected if all the edges are bidirectional. So, to prove the graph is directed, we can show that exists at least one edge that is not bidirectional, i.e. if one key of `graph1` is not contained in `graph2`.

E.g. the node `52` is contained in `graph1`, so there is one edge that starts from this node, but this is not contained in `graph2`, therefore the entire graph is directed.

We can, then, control if there is at least one element that is not contained in both keys' list.

In [4]:
sorted(graph1.keys()) == sorted(graph2.keys()) 

False

- The number of **nodes** is the union of the sets that contain the unique keys for both `graph1` and `graph2`.

In [5]:
V = len(set.union(set(graph1.keys()),set(graph2.keys())))
V

461193

- The number of **edges** is the length of the dataset `reduced`.

In [6]:
E = len(reduced) 
E

2645247

- **Average** node **degree** and **density**: for a directed graph the average degree is the number of edges $E$ divided by the number nodes $V$.

In [7]:
avg_degree = E/V 
avg_degree

5.735661642739591

The graph is not dense. 

To compute the density in a directed graph: $D={\frac{|E|}{|V|\,(|V|-1)}}$, where $E$ is the number of edges and $V$ is the number of vertices in the graph.

In [8]:
density = E/(V*(V-1))
density

1.2436602635647606e-05

## [RQ2]
### Categories ranking
We create the graph `G` using the function `from_pandas_edgelist` from the package *NetworkX* and the dictionary `G_dict` with the following structure:
- key: the nodes of the graph;
- value: the neighbors of the node.

In [9]:
G = nx.from_pandas_edgelist(reduced, 0, 1, create_using = nx.DiGraph()) 
G_dict = {node:{i for i in G.neighbors(node)} for node in G.nodes} 

After the download of the `wiki-topcats-categories.txt`, we create a dictionary `cats` where:
- key: the name of the categories;
- value: the id of the page belonging to the category.

The dictionary is cleaned by turning the strings of the page id's into integers and then filtered by whether they are in `G_dict`.

In [10]:
cats = {}

with open('wiki-topcats-categories.txt','r') as categories:
    for line in categories:
        pages = line[line.index(';')+2:].split(' ') # List of pages id's as strings
        pages = [int(page.replace('\n','')) for page in pages if page.isdigit() == True]
        
        if len(pages) >= 3500:
            # The name of the category is in between the : and ;
            cats[line[line.index(':')+1:line.index(';')]] = [page for page in pages  if page in G_dict]

Here we define our version of the **Breadth First Search** algorithm: it keeps track of the number of steps it is needed to get from the starting node to any other node of the graph.

In [11]:
def bfs(graph, start):
    visited = set() # Set of the visited nodes
    queue = [start] # List of unvisited nodes
    levels_of_nodes = {} # -key: node -value: number of steps from start to node
    current_level = 0
    
    while queue: # While there are unvisited nodes
        neig_list = [] # List of neighbours of the nodes
        for node in queue:
            if node not in visited:
                visited.add(node)
                neighbours = graph[node]
                neig_list.extend(neighbours - visited) # Put on `neig_list` only the unvisited neighbours 
                levels_of_nodes[node] = current_level # Mapping to node the number of steps
        # When the nodes in queue are all passed in the for loop, they are not deleted from the queue  
        # so, we have to reinitialize it
        queue = [] 
        queue.extend(neig_list)
        current_level += 1
            
    return levels_of_nodes

We let the user choose category $\text{C}_0$ from the list of categories. 

In [12]:
C_0 = input()

English_footballers


We can now apply the `bfs` function to the nodes of the input category and we append the returned dictionaries of nodes and number of steps to a list.

In [13]:
lst_of_dicts = []

for node in tqdm(cats[C_0]): 
    levels_of_nodes = bfs(G_dict, node)
    lst_of_dicts.append(levels_of_nodes)

100%|██████████| 7537/7537 [1:22:26<00:00,  1.37it/s]


Here, a new dictionary `shortest_paths` is created, containing the nodes and their relative shortest path from every node of the input category.

In [14]:
shortest_paths = {}

for dic in lst_of_dicts:
    for node, level in dic.items():
        if node not in shortest_paths:
            shortest_paths[node] = [level] 
        else:
            shortest_paths[node].append(level)

The dictionary `cats_paths` containes the set of all the possible shortest paths between the nodes of category $\text{C}_0$ and the remaining.

In [15]:
cats_paths = {cat:[] for cat in cats if cat != C_0} 

for cat in cats:
    if cat != C_0:
        for node in cats[cat]:
            if node in shortest_paths:
                cats_paths[cat].extend(list(filter(lambda x: x != 0, shortest_paths[node])))

Now, for every category the median of the shortest paths is computed using the function `median` from the package *statistics* and divided by the lenght of the amount of the nodes that are not connected to the others. Then the categories are sorted by the distance from the input category obtained in this way.

In [16]:
median_dict = {}

for cat, paths in cats_paths.items():
    median_dict[cat] = statistics.median(paths) / len(str(len(cats[cat])*len(cats[C_0])-len(cats_paths[cat])))
    
median_lst = sorted(median_dict.items(), key=itemgetter(1))

Finally, this is the resulting order.

In [17]:
block_rank = [C_0] + [item[0] for item in median_lst]
block_rank

['English_footballers',
 'Living_people',
 'The_Football_League_players',
 'Association_football_forwards',
 'Association_football_midfielders',
 'Article_Feedback_Pilot',
 'Association_football_goalkeepers',
 'Association_football_defenders',
 'Harvard_University_alumni',
 'Major_League_Baseball_pitchers',
 'English_cricketers',
 'English-language_albums',
 'English-language_films',
 'American_films',
 'Fellows_of_the_Royal_Society',
 'People_from_New_York_City',
 'American_television_actors',
 'American_film_actors',
 'Black-and-white_films',
 'Place_of_birth_missing_(living_people)',
 'American_military_personnel_of_World_War_II',
 'Year_of_birth_missing_(living_people)',
 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies',
 'English_television_actors',
 'British_films',
 'American_Jews',
 'Year_of_birth_unknown',
 'Year_of_death_missing',
 'Rivers_of_Romania',
 'Debut_albums',
 'Year_of_birth_missing',
 'Windows_games',
 'Indian_films',
 'Main_Belt_asteroids',
 '

Now, we clean the dictionary of the categories and pages `cats`: if a page is contained in more than one category, it will stay in the closest one to the input category while the others will be removed.

In [18]:
visited = set()

for cat in block_rank:
    for node in cats[cat]:
        if node not in visited:
            visited.add(node)
        else:
            cats[cat].remove(node)

Once obtained the `block_rank` vector, we want to order the nodes in each category.

### Nodes ranking

First of all, we create the nested dictionary `counters_of_cats` that maps every category to a dictionary of nodes and respective weight computed summing the number of the in-edges belonging to the same category.

In [19]:
counters_of_cats = {}

for i in range(len(block_rank)):
    nodes_C_i = [node for node in cats[block_rank[i]]]
    # The `subgraph` function computes the subgraph induced by the category C_i of the graph G
    sub_graph = G.subgraph(nodes_C_i)
    # `sub_graph_dict` is the dictionary that associates to every node its neighbors
    sub_graph_dict = {node: [neig for neig in sub_graph.neighbors(node)] for node in nodes_C_i}
    # `flat_lst` contains the nodes repeated for every time they are neighbors of other nodes
    flat_lst = [neig for set_ in sub_graph_dict.values() for neig in set_]
    # `dict_weight_C_i` counts the number of times the node is a neighbor of other node
    dict_weight_C_i = Counter(flat_lst)
    counters_of_cats[block_rank[i]] = dict_weight_C_i

The dictionary `pred_dict` has the following structure:
- key: node of the graph $G$;
- value: a set of the predecessors of the node.

For every node in `counters_of_cats`, if a predecessor of it is in the previous category, its weight is added to the node's one.

In [20]:
pred_dict = {node:{pred for pred in G.predecessors(node)} for node in G.nodes}

for i in range(1, len(block_rank)): # Start from the second category
    for node in counters_of_cats[block_rank[i]]:
        for pred in pred_dict[node]:
            counters_of_cats[block_rank[i]][node] += counters_of_cats[block_rank[i-1]][pred] # Adding the weight     

The results are then sorted by their weight.

In [21]:
result = {}

for cat,rank_dict in counters_of_cats.items():
    rank = sorted(rank_dict.items(), key=itemgetter(1), reverse=True)
    result[cat] = rank

Since every node in `counters_of_cats` corresponds to a certain page, we open the file `wiki-topcats-page-names.txt` and we extract the page names.

Finally, we show the first three most ranked pages for every category.

In [22]:
page_names = {}

with open('wiki-topcats-page-names.txt', 'r') as f:
    for line in f.readlines():
        page_names[line[:line.index(' ')]] = line[line.index(' ')+1:].replace('\n', '')

df_result = pd.DataFrame(index=list(result.keys()), columns=['First', 'Second', 'Third'])
for i,cat in enumerate(result):
    first_3 = result[cat][:3]
    for j, tuple_ in enumerate(first_3):
        df_result.iloc[i][j] = page_names[str(tuple_[0])]
        
df_result

,First,Second,Third
English_footballers,Glenn Hoddle,Terry Venables,Brian Clough
Living_people,George W. Bush,Barack Obama,Bill Clinton
The_Football_League_players,Kevin Keegan,Glenn Hoddle,Kenny Dalglish
Association_football_forwards,Kenny Dalglish,Alan Shearer,Robbie Fowler
Association_football_midfielders,David Beckham,George Best,Roy Keane
Article_Feedback_Pilot,Arsenal F.C.,Sky Sports,West Ham United F.C.
Association_football_goalkeepers,Goalkeeper (association football),ukasz Fabiaski,Manuel Almunia
Association_football_defenders,Sbastien Squillaci,Laurent Koscielny,Fernando Hierro
Harvard_University_alumni,John F. Kennedy,Theodore Roosevelt,John Adams
Major_League_Baseball_pitchers,Babe Ruth,Sandy Koufax,Cy Young
